In [1]:
%matplotlib inline
import pandas as pd
import os
import re
import shutil
import numpy as np
import warnings

Duplicate key in file WindowsPath('C:/Users/maa/.matplotlib/stylelib/paper.mplstyle'), line 461 ('figure.figsize   : 5, 3   ## figure size in inches')
Duplicate key in file WindowsPath('C:/Users/maa/.matplotlib/stylelib/paper.mplstyle'), line 462 ('figure.dpi       : 100        ## figure dots per inch')


In [2]:
df = pd.read_csv('students.csv')

In [3]:
df

,family name,name
0,Dahlén,Isak
1,Dahlqvist,Petrus
2,Danielsson,Isac
3,Durkin,Kirill
4,Dynevik,Philip
5,Edsbagge,Carl
6,Egeland,Karl
7,Ekdahl,Jonathan
8,El Masri,Mohammed Ali
9,Elm Jonsson,Benjamin


In [4]:
regexp=re.compile(r'^ *(.*)')
def clean_names(name):
    result=regexp.search(name)
    return result.group(1)

In [5]:
df['name']=df['name'].apply(func=clean_names)

In [6]:
def create_folder(row,root_path='assignments'):
    
    family_name = row['family name']
    name = row['name']
    dir_name = '%s_%s' % (family_name,name)
    path = os.path.abspath(os.path.join(root_path,dir_name))
    if not os.path.exists(path):
        os.mkdir(path)
    
    return path
    

## Create folders:

In [7]:
root_path='assignments'

if not os.path.exists(root_path):
    os.mkdir(root_path)

df['dir_path'] = df.apply(func=create_folder, axis=1, root_path=root_path)

In [8]:
def make_clickable(val):
    s=r'file:///' + val
    return '<a href="{}">{}</a>'.format(s,"path")

In [9]:
def show(df):
    return df.style.format({'dir_path': make_clickable})

## Check exist: *student_AutomaticGearbox.m*

In [10]:
run_file_name='student_AutomaticGearbox.m'
def has_run_file(row, run_file_name='student_AutomaticGearbox.m'):
    path=os.path.join(row['dir_path'],run_file_name)
    return os.path.exists(path)

In [11]:
def recreate_run_file(row):
    files = os.listdir(row['dir_path'])
    for file in files:
        name,ext = os.path.splitext(file)
        if ext=='.m':
            if 'student_AutomaticGearbox' in file:
                src=os.path.join(row['dir_path'],file)
                dst=os.path.join(row['dir_path'],'student_AutomaticGearbox.m')
                if not os.path.exists(dst):
                    shutil.copyfile(src,dst)

In [12]:
df['has_run_file'] = df.apply(func=has_run_file, axis=1)

In [13]:
mask = ~df['has_run_file']
df.loc[mask].apply(func=recreate_run_file, axis=1)

,family name,name,dir_path,has_run_file


In [14]:

show(df.loc[mask])

,family name,name,dir_path,has_run_file


## Running the scripts to get the times

In [15]:
def runner(row, eng):
    
    print('evaluate: %s %s' % (row['name'],row['family name']))
    run_script_name = 'student_AutomaticGearbox.m'
    
    if os.path.exists(run_script_name):
        os.remove(run_script_name)  # Remove old.
    
    assert not os.path.exists(run_script_name)
    
    for file in os.listdir(row['dir_path']):
        ext = os.path.splitext(file)[-1]
        if ext=='.m' or ext=='.mat':
            src=os.path.join(row['dir_path'],file)
            dst=file
            shutil.copyfile(src,dst)
    
    assert os.path.exists(run_script_name)
    
    result = eng.runner()
    finishing_time=result['TimeTotal'][0][-1]
    return finishing_time
    

In [ ]:
import matlab.engine
eng = matlab.engine.start_matlab()

for index, row in df.iterrows():
    
    try:
        finishing_time = runner(row=row, eng=eng)
    except Exception as e:
        fail = True
        df.loc[index,'finishing_time'] = np.NaN
        print('Fail!')
    else:
        fail = False
        df.loc[index,'finishing_time'] = finishing_time
            
eng.quit()

evaluate: Isak Dahlén
evaluate: Petrus Dahlqvist
evaluate: Isac Danielsson
evaluate: Kirill Durkin
evaluate: Philip Dynevik
evaluate: Carl Edsbagge
evaluate: Karl Egeland
evaluate: Jonathan Ekdahl
evaluate: Mohammed Ali El Masri
evaluate: Benjamin Elm Jonsson
evaluate: Johan Elnertz
evaluate: Martin Engelsen
evaluate: Love Engh
evaluate: Daniel Eriksson
evaluate: Vigg Eriksson
evaluate: Johan Erlandsson
evaluate: Jonathan Erneborg
evaluate: David Espedalen
evaluate: Fabian Fagerlind
evaluate: Hilda Fahl
evaluate: Bekir Fazlija
evaluate: Rasmus Fischer
Fail!
evaluate: Sofia Friberg
Fail!
evaluate: Emelie Gillerstedt
Fail!
evaluate: Milton Göransson
Fail!
evaluate: Anna Götberg
Fail!
evaluate: Frida Gotenius


In [ ]:
eng.quit()

In [ ]:
df